# Your First MLOps Pipeline

## Aim

> Build several common components found in MLOps pipelines to illustrate the essential concepts of MLOps.


This is by no means a comprehensive overview of all facets of MLOps, but it aims to illustrate a few of the important concepts.

## Outline
- What is MLOps?
- Turning the ML model training process into a pipeline
- Artefact and metadata tracking
  - Tensorboard demo
- Serving
  - APIs
- Drift
  - Data drift
  - Concept drift
- Monitoring
  - Visualising data drift
  - Logging user requests to visualise
- Alerting
  - Pagerduty example
- Retraining
  - Collecting more data to train on
  - Cron
  - Automatic retraining

## What is MLOps?

- MLOps is shorthand for Machine Learning Operations. 
- MLOps is everything other than the model training code required to put AI systems into production
- MLOps is to AI engineering what devops is to software engineering.
- MLOps empowers organisations not just to deploy once, but to deploy over and over again quickly and efficiently by reducing the overhead and increasing automation that usually goes into maintaining machine learning systems in production.

## Why do we need MLOps?

- Serve more models
- Serve new versions of the same model

Why do we need new models?
- Unexpected performance differences between testing and production
- Evolving environments


### About the dataset

In this example, we're working with data from an online retailer, like Amazon, who make timely offers to their customers after every purchase. We want to build a machine learning model to determine the likelihood of a user claiming the offer so that we can confidently offer it to people who will take it. We don't want to offer it to everyone because it costs us to make the offer, which reduces our margins.

- Features:
    - `product_rating`: the difference between the average rating for that product and the user's rating
    - `delivery_duration`: the difference between the claimed delivery time and the actual delivery time
- Label:
    - `used_offer`: Whether the user claimed an offer shared with them after their successful delivery

However, we know that the distribution of data might change over time:
- Buyers may become more or less sensitive to product quality as supply changes
- Buyers may become more or less sensitive to delivery duration
- Buyers may change whether they claim the offer or not due to changes in the economy

> _Note: The datapoints are ordered sequentially in time, so you can see how the data changes over time_

In [38]:
import pandas as pd


def load_data():
    data = pd.read_csv('data/initial_data.csv')
    # ^^^ could read from database, filesystem storage, or other source in another application

    features = data.drop(columns=["used_offer"])
    labels = data["used_offer"]

    return features, labels


features, labels = load_data()

print(features.describe())
print(features.head())
print(labels.head())


       product_rating  delivery_duration
count      971.000000         971.000000
mean         0.848218           1.245125
std          1.187715           1.302528
min         -3.151794          -2.930829
25%          0.056051           0.352596
50%          0.820649           1.247354
75%          1.708110           2.196187
max          3.964207           5.192221
   product_rating  delivery_duration
0       -1.435952          -0.934353
1        0.850237           1.462074
2        0.212885           0.326928
3        0.172982          -0.057004
4        0.170494          -1.584580
0    0
1    0
2    0
3    1
4    1
Name: used_offer, dtype: int64


## What goes into training a machine learning model?

You could fill a library with everything that I could mention here, but to summarise it, here are the key steps that go into training a ML model:
- Data preparation
- Model training
- Hyperparameter tuning
- Validation set evaluation
- Test set evaluation

The cells below illustrate what a simple version of this might look like.

In [39]:
# split
from sklearn.model_selection import train_test_split

def split_data(features, labels):
    features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, random_state=42) # split data into train and test data
    features_train, features_val, labels_train, labels_val = train_test_split(features_train, labels_train, test_size=0.2, random_state=42) # split train data into train and validation data
    return features_train, features_test, features_val, labels_train, labels_test, labels_val

features_train, features_test, features_val, labels_train, labels_test, labels_val = split_data(features, labels)

Most machine learning algorithms learning procedure is controlled by some parameters that are set before the learning takes place during what is known as model training. These parameters set before training are called hyperparameters.

The cell below defines a function that gets some random hyperparameters. You can tweak these around to try out different model configurations.

I won't go into it here, but in practice, you'd want to systematically sample a range of hyperparameters and use the performance on the validation set to determine which performs best on unseen data.

We'll use a decision tree classifier as our machine learning model. It's a simple model that can still work remarkably well. You can read about it's hyperparameters [here](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

In [40]:
def get_hyperparameters():
    return {
        'max_depth': 4,
        'min_samples_split': 2,
        'min_samples_leaf': 1,
    }

hyperparameters = get_hyperparameters()

Now let's train the model

In [41]:
from sklearn.tree import DecisionTreeClassifier

def train_model(features, labels, hyperparameters):
    """Trains a model on the data"""

    model = DecisionTreeClassifier(
        **hyperparameters
    )
    model.fit(features, labels)



    return model

model = train_model(features_train, labels_train, hyperparameters)

Next, we need to evaluate the performance of the model

In [42]:
# implement methods for the different parts of the pipeline, including hyperparameter tuning and evaluation

from sklearn.metrics import accuracy_score

def evaluate_model(model, X_test, y_test):
    """Evaluates the model on the test set"""

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

accuracy = evaluate_model(model, features_test, labels_test)

print(f"Accuracy: {accuracy}")



Accuracy: 0.9641025641025641


Now create a function to save the model

In [43]:
# save model
import joblib

def save_model(model):
    """Saves the model to disk"""

    joblib.dump(model, "model.joblib")

save_model(model)

Putting that all together...

In [44]:

features, labels = load_data()
train_features, validation_features, test_features, train_labels, validation_labels, test_labels = split_data(features, labels)
hyperparameters = get_hyperparameters()
model = train_model(train_features, train_labels, hyperparameters)
accuracy = evaluate_model(model, test_features, test_labels)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9423076923076923


### Pipelining the ML training process

Imagine you're a data scientist who's developed a machine learning model. You've found out how to create a model that works by setting the right model configuration and processing the data correctly. However, you know that in the future your data is going to change, because you know that over time, the inputs will change with trends. That means you're going to train this model more than once, which is why it's useful to keep this code for re-use.

Now, we can put that all into a function that trains the model from end to end.

In [45]:
def train_and_evaluate_model(features, labels):
    features, labels = load_data()
    train_features, validation_features, test_features, train_labels, validation_labels, test_labels = split_data(features, labels)
    hyperparameters = get_hyperparameters()
    model = train_model(train_features, train_labels, hyperparameters)
    accuracy = evaluate_model(model, test_features, test_labels)
    return model, accuracy, hyperparameters

train_and_evaluate_model(features, labels)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9423076923076923


### End-to-end Pipeline Tools for Model Training used in the Wild

- Airflow
- AWS Sagemaker
- Databricks
- Kubeflow

## Metadata Tracking

### What is model metadata?
Model metadata refers to information about a machine learning model that is not part of the actual model itself, but rather provides context and descriptive details about the model. This metadata can include information such as the model's name, version, author, creation date, input and output formats, hyperparameters, performance metrics, and more.

### Why is it important?
Metadata is important because it allows users to understand the characteristics of a model, how it was created, and how it can be used. This information can help users make informed decisions about which models to use for their particular tasks, and can also aid in reproducibility and collaboration between data scientists and other stakeholders.


### Implementation
The worst way to keep track of your model metadata would be to build your own solution. Everyone's had that idea, and there are simply many way better tools than what you can use off the shelf.

Model metadata can be stored and accessed in various ways, such as through a separate file or database that accompanies the model, or as part of the model's documentation or comments within the code itself. Some machine learning frameworks and platforms also provide tools for automatically generating and managing model metadata.

In this illustration, I'll use Tensorboard to track model metadata. It's in no way the most sophisticated tool for the job - there are many other more advanced solutions used in the wild that I'll share after the following code. Check out the tensorboard documentation [here](https://pytorch.org/docs/stable/tensorboard.html#torch.utils.tensorboard.writer.SummaryWriter.add_hparams).

Let's update our training pipeline to log some of the important metadata. Once you run this code, you'll notice a `runs` folder appear, which contains the metadata - it's simply saved in filesystem storage.

_Bonus points if you can implement it with a [decorator](https://www.youtube.com/watch?v=fm_oY5tXD_s)_

In [46]:
# log metadata using tensorboard
from torch.utils.tensorboard import SummaryWriter

def log_metadata(model, accuracy, hyperparameters):
    """Logs metadata to tensorboard"""

    writer = SummaryWriter()
    writer.add_hparams(
        hparam_dict=hyperparameters, 
        metric_dict={"accuracy": accuracy}
    )
    writer.close()

log_metadata(model, accuracy, hyperparameters)

### Model Metadata Tracking Tools used in the Wild

- Weights and biases
- MLFlow
- Neptune

## Serving

Now that the model has been trained, the easy part is over. The next step is serving predictions to users. Serving is the essence of what people mean when they talk about "deployment".

The most common pattern is to serve your model predictions through an API. Users can make requests to the API and receive responses. E.g. Someone can make a request for a prediction, and we serve them that prediction by processing their data with our model.

Below is the Python code that defines the API endpoints that could serve the model.

_Note: The last line, which sets the API up to listen for requests, won't work in a notebook because it needs to be run from a Python file. However, the methods above will still be defined, so make sure to run this cell because we'll be using those methods going forward._

In [47]:
from fastapi import FastAPI
import uvicorn

model = joblib.load("model.joblib")  # load model from disk

api = FastAPI()

@api.get("/")
def root():
    return {"Hello": "World"}

@api.get("/predict") # defines the /predict endpoint
def predict(data): # defines that the endpoint takes a parameter called data

    data = pd.read_json(data)
    prediction = model.predict(data)
    return prediction.tolist()


uvicorn.run(api, host='localhost', port=8000) # UNCOMMENT THIS TO RUN IN A PYTHON FILE

RuntimeError: asyncio.run() cannot be called from a running event loop

To deploy the API defined in the above cell, here's what you would do with the code:
1. Put into a Python file
1. Define a Docker image that contains that file, the model params, and the model class
1. Run that Docker container from that image on a computer in the cloud

From another file or application, you could make requests to the API like this:

_Note: Like the cell above, we expect an error to be thrown, because we can't run an API from a notebook, so the endpoint won't be found when we try to make a request._

In [48]:
import requests
import json

API_ROOT = "http://localhost:8000"
API_ENDPOINT = "/predict"
URL = API_ROOT + API_ENDPOINT

features = pd.DataFrame([
    {
        "product_rating": 4.5,
        "delivery_duration": 0.2,
    },
    {
        "product_rating": 1.5,
        "delivery_duration": 0.1,
    },
])

payload = features.to_json()

response = requests.get(URL, data=payload)


ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f957f8478b0>: Failed to establish a new connection: [Errno 61] Connection refused'))

To simplify the scenario for use in a notebook, 

IGNORE THIS CELL OR RISK MISUNDERSTANDING

In [59]:
features = pd.DataFrame([
    {
        "product_rating": 4.5,
        "delivery_duration": 0.2,
    },
    {
        "product_rating": 1.5,
        "delivery_duration": 0.1,
    },
])

payload = features.to_json()
predict(payload)


[1, 1]

We get back two numbers that represent the model's predictions for the two examples we sent with our request.

### API Clients

In our case, here's a simulated example version that works from within a notebook. Often, many APIs, like the OpenAI API for example, come packaged in a Python library. When an API is accessible through a library in a programming language, we call that library a _client_. The client makes the API easy to use because calling the method covers up the direct HTTP request under the hood.

_Note: The below code is not something that you would have in a real world situation, it simply poses as an API for us here, because we can't run one through a notebook_

In a real situation, an API client's source code might look like this:

In [50]:
class APIClient:
    def __init__(self, api_root):
        self.api_root = api_root

    def predict(self, data):
        payload = data.to_json()
        response = requests.get(self.api_root + "/predict", data=payload)
        return response.json()

In [57]:
class APIClient:
    def __init__(self, model):
        self.model = model

    def predict(self, data):
        data = data.to_json()
        prediction = predict(data) # IN PRACTICE, THIS LINE WOULD BE A REQUEST TO A REMOTE API (see first APIClient class)
        return prediction


Below shows how you would typically use an API client

In [58]:
api_client = APIClient(model)
api_client.predict(features)

[1, 1]

### Serving Tools used in the Wild

- FastAPI
- AWS Sagemaker
- KFServing
- Kubernetes
- Docker (underpins a lot of the above)

## Monitoring

You can't run ML models in production without keeping an eye on them.

### Monitoring Tools used in the Wild
- Prometheus
- Grafana
- Evidently
- Fiddler
- Arize

Let's start off by logging some basic hardware metrics, like [CPU utilisation](psutil.cpu_percent(4)) using the Python library `psutil`

In [ ]:
!pip install psutil

In [86]:
# Importing the library
import os
import psutil

def log_hardware_metrics():
    cpu_utilisation_percent = psutil.cpu_percent(4)
    print(f"CPU utilisation: {cpu_utilisation_percent}%")

    load1, load5, load15 = psutil.getloadavg() # get the load average over the last 1, 5 and 15 minutes
    cpu_usage = round(100 * load15/os.cpu_count(), 1)

    print(f"15 minute rolling average CPU utilisation: {cpu_usage}%")


CPU utilisation: 13.5%
15 minute rolling average CPU utilisation: 39.8%


Let's add that hardware metric logging to our API.

In [ ]:
class SimulatedAPIClient:
    def __init__(self, model):
        self.model = model

    def predict(self, data):
        data = pd.read_json(data)
        prediction = self.model.predict(data)
        return prediction.tolist()


As well as these hardware metrics though, there are some things that you should track specific to ML.

## Drift - Is my problem changing over time?

Run the cell below to define an object that simulates the data changing over time